# Python Refresher Lab — FAANG-Level Hands-On

**Goal:** Refresh Python fundamentals + interview patterns the way FAANG expects ML engineers to code.

**Outcome:** You can write clean, efficient Python, explain complexity, and avoid classic interview gotchas.

---

## How to Start
1. Open this notebook in Colab
2. Complete all **TODO** sections
3. Restart runtime → Run all

---

## Lab Rules (FAANG Style)
- ✅ Complexity for each core function
- ✅ Handle edge cases
- ✅ Prefer O(n) over O(n²) when feasible
- ✅ Add tests
- ❌ No hardcoding outputs


In [19]:
from __future__ import annotations
from collections import Counter, defaultdict, deque
import heapq

# NOTE: Use this helper for quick sanity checks
def check(name: str, cond: bool):
    if not cond:
        raise AssertionError(f'Failed: {name}')
    print(f'OK: {name}')

## Section 1 — Python Idioms & Core Data Structures

### Task 1.1: Comprehensions (no unnecessary loops)

Create (without loops):
- `even_squares`: squares of even numbers from 1..20
- `pairs`: list of (x, x*x) for x=1..10
- `flattened`: flatten `nested`

**Explain:** Why can comprehensions be faster than manual loops?

**Answer:** Comprehensions can be faster than manual loops because Python can do the work more efficiently behind the scenes. They use fewer steps, avoid repeated method calls, and run more of the logic in optimized internal code. This reduces extra work for Python, so the result is often produced more quickly.


In [20]:
# TODO:
# Create even_squares
# HINT: range(1, 21), filter even with x % 2 == 0
even_squares = [x*x for x in range (1,21) if x % 2 == 0]

# TODO:
# Create pairs
pairs = [(x, x * x) for x in range (1,11)]


# TODO:
# Flatten nested
#nested = [[1, 2], [3, 4], [5, 6]]
# HINT: nested loops inside a single comprehension
flattened = [y for x in nested for y in x]

check('even_squares', even_squares == [4, 16, 36, 64, 100, 144, 196, 256, 324, 400])
check('pairs', pairs[:3] == [(1, 1), (2, 4), (3, 9)] and pairs[-1] == (10, 100))
check('flattened', flattened == [1, 2, 3, 4, 5, 6])

OK: even_squares
OK: pairs
OK: flattened


### Task 1.2: Counter (frequency counting)

Given `s`, compute:
- character frequency using `Counter`
- top 2 most common characters

**Interview angle:** When is Counter preferable to sorting?

**Answer:** Counter is preferable to sorting when you only need to count how often items appear, not arrange them in order. It scans the data once and keeps track of counts, which is faster and simpler than sorting, especially for large datasets.

In [21]:
s = 'mississippi'

# TODO:
freq = Counter(s)
top2 = freq.most_common(2)

print(freq)
print(top2)
check('top2', top2[0][0] == 'i' and top2[0][1] == 4)

Counter({'i': 4, 's': 4, 'p': 2, 'm': 1})
[('i', 4), ('s', 4)]
OK: top2


### Task 1.3: defaultdict (grouping)

Group words by first letter.

**HINT:** `defaultdict(list)`

**Gotcha:** Don’t use a normal dict unless you handle missing keys.

In [22]:
words = ['apple', 'apricot', 'banana', 'blueberry', 'cherry', 'avocado']

# TODO:
groups = defaultdict(list)
for word in words:
    groups[word[0]].append(word)

print(groups)

print(dict(groups))
check('groups[a]', dict(groups)['a'] == ['apple', 'apricot', 'avocado'])

defaultdict(<class 'list'>, {'a': ['apple', 'apricot', 'avocado'], 'b': ['banana', 'blueberry'], 'c': ['cherry']})
{'a': ['apple', 'apricot', 'avocado'], 'b': ['banana', 'blueberry'], 'c': ['cherry']}
OK: groups[a]


## Section 2 — FAANG Core Patterns

### Task 2.1: Two Sum (Hash Map Pattern)

**FAANG Classic.** Return indices of two numbers that sum to target.

# HINT:
- Use a dict: value -> index
- For each `x`, check if `target-x` has been seen

**Explain:** Why is this O(n) time?

Answer: The code has **O(n)** time complexity because it goes through the list **only once**. For each number, it checks whether the needed value (`target - num`) is already in the hash map, and this lookup takes **constant time O(1)** on average. Adding a number to the hash map also takes **O(1)** time. Since these constant-time operations happen once for each of the `n` elements in the list, the total time grows linearly with the size of the input, making the overall time complexity **O(n)**.

In [23]:
def two_sum(nums, target):
    # TODO
    hashMap = {}
    for i, num in enumerate(nums):
        if target - num in hashMap:
            return [hashMap[target - num], i]
        hashMap[num] = i

check('two_sum_1', two_sum([2, 7, 11, 15], 9) == [0, 1])
check('two_sum_2', two_sum([3, 2, 4], 6) == [1, 2])
check('two_sum_dup', two_sum([3, 3], 6) == [0, 1])

OK: two_sum_1
OK: two_sum_2
OK: two_sum_dup


### Task 2.2: Longest Substring Without Repeating Characters (Sliding Window)

**FAANG Classic/Gotcha:** naive O(n²) solutions time out.

# HINT:
- Keep a map char -> most recent index
- Maintain a window `[l..r]` with unique chars

**Explain:** What invariant does the window maintain?

Answer: The window always maintains the invariant that **all characters inside it are unique**. At any moment, the substring from index `l` to `r` contains no repeating characters. When a character repeats, the left pointer `l` is moved just past the previous occurrence of that character to restore this property.

In [24]:
def length_of_longest_substring(s: str) -> int:
    # TODO
    last_seen = {}
    l = 0
    best = 0

    for r, ch in enumerate(s):
        if ch in last_seen and last_seen[ch] >= l:
            l = last_seen[ch] + 1
        last_seen[ch] = r
        best = max(best, r - l + 1)

    return best

check('ls_abcabcbb', length_of_longest_substring('abcabcbb') == 3)
check('ls_bbbbb', length_of_longest_substring('bbbbb') == 1)
check('ls_pwwkew', length_of_longest_substring('pwwkew') == 3)
check('ls_empty', length_of_longest_substring('') == 0)

OK: ls_abcabcbb
OK: ls_bbbbb
OK: ls_pwwkew
OK: ls_empty


### Task 2.3: Product of Array Except Self (Prefix/Suffix)

**FAANG Classic/Gotcha:**
- No division
- O(n) time

# HINT:
- Build prefix products in one pass
- Multiply by suffix products in reverse pass

**Explain:** Why does this work with zeros?

Answer: This works with zeros because the algorithm never divides and instead builds products from the left and right separately. When a zero appears, it naturally makes the prefix or suffix product become zero, which correctly forces all positions that depend on that zero to be zero. The one special case—when the current index is the zero itself—still works because the prefix product includes everything to the left and the suffix product includes everything to the right, excluding the zero. As a result, only the index with the zero gets the product of all non-zero values, and all other positions become zero automatically.

In [25]:
def product_except_self(nums):
    # TODO
    n = len(nums)
    res = [1] * n

    prefix = 1
    for i in range(n):
        res[i] = prefix
        prefix *= nums[i]

    suffix = 1
    for i in range(n - 1, -1, -1):
        res[i] *= suffix
        suffix *= nums[i]

    return res


check('pes_1234', product_except_self([1, 2, 3, 4]) == [24, 12, 8, 6])
check('pes_zero', product_except_self([-1, 1, 0, -3, 3]) == [0, 0, 9, 0, 0])

OK: pes_1234
OK: pes_zero


## Section 3 — FAANG Hard (Optional / Bonus)

### Task 3.1: Sliding Window Maximum (Deque)

**FAANG Classic:** maintain a decreasing deque of indices.

# HINT:
- Pop from left if out of window
- Pop from right while current value is larger

**Explain:** Why is this amortized O(n)?

Answer : The algorithm is amortized **O(n)** because each index is added to the deque only once and removed at most once. Although the inner `while` loop may remove several elements at a single step, each element can only be popped from the deque one time over the entire execution. Similarly, elements are removed from the front only when they fall out of the window, which also happens once per index. Since the total number of insertions and removals across the whole process is linear, the overall time complexity remains **O(n)**.


In [26]:
def max_sliding_window(nums, k):
    # TODO
    dq = deque()  # stores indices
    res = []

    for i, num in enumerate(nums):
        # Remove indices that are out of the current window
        if dq and dq[0] <= i - k:
            dq.popleft()

        # Remove smaller values from the right
        while dq and nums[dq[-1]] < num:
            dq.pop()

        dq.append(i)

        # Start recording results once the first window is complete
        if i >= k - 1:
            res.append(nums[dq[0]])

    return res

check('msw', max_sliding_window([1,3,-1,-3,5,3,6,7], 3) == [3,3,5,5,6,7])

OK: msw


### Task 3.2: LRU Cache (Hash Map + Doubly Linked List)

**FAANG Hard/Gotcha:** You must get O(1) `get` and `put`.

# HINT:
- Dict: key -> node
- Doubly linked list: move-to-front on access

You can implement this later if you run out of time today.

---

## Submission Checklist
- All TODOs completed
- All tests passing
- Explain prompts answered
- Complexity noted
